In [4]:
import pandas as pd
import sqlite3

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [30]:
connection = sqlite3.connect("/media/padua/6004B63504B60E52/TCC/database_text_to_sql.db")
file_path = "/media/padua/6004B63504B60E52/TCC/export/THOTH/versao 2/prescricao-tratada.csv"

In [4]:
for chunk in pd.read_csv(file_path, chunksize=10000, encoding="latin1"):
    chunk.to_sql("Presc", connection, if_exists="append", index=False)


In [32]:
cursor = connection.cursor()

In [33]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

In [34]:
tabelas = cursor.fetchall()
print(tabelas)

[('Prescriptions',), ('Presc',)]


In [12]:
cursor.execute(f"PRAGMA table_info({'Presc'});")

columns = cursor.fetchall()
for column in columns:
    print(column)

(0, 'uuid', 'TEXT', 0, None, 0)
(1, 'prescricao', 'INTEGER', 0, None, 0)
(2, 'prontuario', 'INTEGER', 0, None, 0)
(3, 'data_atendimento', 'TEXT', 0, None, 0)
(4, 'idade_anos', 'INTEGER', 0, None, 0)
(5, 'idade_meses', 'INTEGER', 0, None, 0)
(6, 'idade_dias', 'INTEGER', 0, None, 0)
(7, 'procedimento', 'TEXT', 0, None, 0)
(8, 'id_profissional', 'TEXT', 0, None, 0)
(9, 'sequencia', 'INTEGER', 0, None, 0)
(10, 'itemseq', 'INTEGER', 0, None, 0)
(11, 'item', 'INTEGER', 0, None, 0)
(12, 'descricao', 'TEXT', 0, None, 0)
(13, 'apresentacao', 'TEXT', 0, None, 0)
(14, 'unidade', 'TEXT', 0, None, 0)
(15, 'dose_qtde', 'REAL', 0, None, 0)
(16, 'dose_unidade', 'TEXT', 0, None, 0)
(17, 'via', 'TEXT', 0, None, 0)
(18, 'intervalo', 'TEXT', 0, None, 0)
(19, 'sequencia_horario', 'INTEGER', 0, None, 0)
(20, 'aprazado', 'TEXT', 0, None, 0)
(21, 'observacao', 'TEXT', 0, None, 0)
(22, 'esquema', 'TEXT', 0, None, 0)
(23, 'velocidade', 'TEXT', 0, None, 0)
(24, 'tipo_prescricao', 'TEXT', 0, None, 0)
(25, 'data_t

In [8]:
cursor.execute(f"SELECT * FROM {'Prescriptions'} LIMIT 5;")
result = cursor.fetchall()

for r in result:
    print(r)

('e2a8a5f0-0090-4393-9c74-4af16c2fac1e', 1318746, 10163814, '2015-08-29 00:00:00.000', 65, 6, 6, '303060212#TRATAMENTO DE INSUFICIENCIA CARDIACA ', '37070872-2e39-4aec-a420-a4334b688bc1', 0, 5, 17501, 'DipiRONA 500 mg', 'COMP', 'COMP', 2.0, 'COMP', 'VO', 'SE NECESSARIO', 0, 'S', 'ATÉ DE 6/6 HORAS, SE DOR OU TAX >38,3ºC ', ' ', ' ', '1', '2016-01-16 12:24:00', 17501, 'DipiRONA 500 mg', 'COMP', 'COMP', 2.0, 'COMP', 'VO', 'SE NECESSARIO', 0, 1, 2.0, None, None, None, None, None)
('3bc7d6cc-650c-47fe-b840-4cea6a3c2a60', 1318746, 10163814, '2015-08-29 00:00:00.000', 65, 6, 6, '303060212#TRATAMENTO DE INSUFICIENCIA CARDIACA ', '37070872-2e39-4aec-a420-a4334b688bc1', 0, 5, 17501, 'DipiRONA 500 mg', 'COMP', 'COMP', 2.0, 'COMP', 'VO', 'SE NECESSARIO', 0, 'S', 'ATÉ DE 6/6 HORAS, SE DOR OU TAX >38,3ºC ', ' ', ' ', '1', '2016-01-16 12:24:00', 17501, 'DipiRONA 500 mg', 'COMP', 'COMP', 2.0, 'COMP', 'VO', 'SE NECESSARIO', 0, 1, 2.0, None, None, None, None, None)
('e2a8a5f0-0090-4393-9c74-4af16c2fac1e

In [6]:
model_name = "Sumitp/Text_to_SQL_BART_sumit1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
# Example of Code Representation Prompt

input_text = """Given the following database schema:
CREATE TABLE Prescriptions (
    uuid TEXT,
    prescricao INTEGER,
    prontuario INTEGER,
    data_atendimento TEXT,
    idade_anos INTEGER,
    idade_meses INTEGER,
    idade_dias INTEGER,
    procedimento TEXT,
    id_profissional TEXT,
    sequencia INTEGER,
    itemseq INTEGER,
    item INTEGER,
    descricao TEXT,
    apresentacao TEXT,
    unidade TEXT,
    dose_qtde REAL,
    dose_unidade TEXT,
    via TEXT,
    intervalo TEXT,
    sequencia_horario INTEGER,
    aprazado TEXT,
    observacao TEXT,
    esquema TEXT,
    velocidade TEXT,
    tipo_prescricao TEXT,
    data_timestamp TEXT,
    item_prescrito INTEGER,
    descricao_prescrita TEXT,
    apresentacao_prescrita TEXT,
    unidade_prescrita TEXT,
    dose_qtde_prescrita REAL,
    dose_unidade_prescrita TEXT,
    via_prescrita TEXT,
    intervalo_prescrito TEXT,
    sequencia_horario_2 INTEGER,
    qtde_esquema INTEGER,
    qtde_total REAL,
    data_atendimento_1 REAL,
    idade_anos_1 REAL,
    idade_meses_1 REAL,
    idade_dias_1 REAL,
    procedimento_1 REAL,
    PRIMARY KEY (uuid)
);

Answer the following question:
What is the total number of prescriptions in the database?
"""
input_ids = tokenizer.encode(input_text, return_tensors="pt")

output = model.generate(input_ids, max_length=200, num_return_sequences=1)

In [19]:
# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

SELECT COUNT Episodes INTEGER FROM table WHERE Dos_HERE ISSECCao_Prescrita Text,                 = via TEXT,                                                ���_Ceremony INTEGG,                 =                ������������������������������������ FROM table =��������������������� =�������������ഐഔാ������������������� =ന�������౐����  FROM (% RegardlesstaboolainventoryQuantity


In [27]:

input_text = """Below is an instruction that describes a task, paired with an input that provides further context . Write a response that appropriately completes the request.
### Instruction :
Write a sql to answer the question "What is the total number of prescriptions in the database?"

### Input :
prescriptions(uuid, prescricao, prontuario, data_atendimento, idade_anos, idade_meses, idade_dias, procedimento, id_profissional, sequencia, itemseq, item, descricao, apresentacao, unidade, dose_qtde, dose_unidade, via, intervalo, sequencia_horario, aprazado, observacao, esquema, velocidade, tipo_prescricao, data_timestamp, item_prescrito, descricao_prescrita, apresentacao_prescrita, unidade_prescrita, dose_qtde_prescrita, dose_unidade_prescrita, via_prescrita, intervalo_prescrito, sequencia_horario_2, qtde_esquema, qtde_total, data_atendimento_1, idade_anos_1, idade_meses_1, idade_dias_1, procedimento_1)
Consider uuid as the primary key
"""
input_ids = tokenizer.encode(input_text, return_tensors="pt")

output = model.generate(input_ids, max_length=300, num_return_sequences=1)

In [28]:
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

SELECT Prescriptions(uuid, prescricao, prontuario, data_atendimento, IDade_anos, idade_dias, ecces, procedimento FROM table WHERE September, 1 `` 1 `` 0 `` 0 `6 = Docte_prescrita, dose_� =��quema, qtde_796Consider uuid as the primary key = Tipo_Prescricao AND ID ID ID = IDade, esquema , Velocidade, Koto_prescicoao AND Templer = data_timestamp,


In [25]:
input_text = """### Complete sqlite SQL query only and with no explanation
### SQLite SQL tables , with their properties :
prescriptions(uuid, prescricao, prontuario, data_atendimento, idade_anos, idade_meses, idade_dias, procedimento, id_profissional, sequencia, itemseq, item, descricao, apresentacao, unidade, dose_qtde, dose_unidade, via, intervalo, sequencia_horario, aprazado, observacao, esquema, velocidade, tipo_prescricao, data_timestamp, item_prescrito, descricao_prescrita, apresentacao_prescrita, unidade_prescrita, dose_qtde_prescrita, dose_unidade_prescrita, via_prescrita, intervalo_prescrito, sequencia_horario_2, qtde_esquema, qtde_total, data_atendimento_1, idade_anos_1, idade_meses_1, idade_dias_1, procedimento_1)
### How many continents are there ? """

input_ids = tokenizer.encode(input_text, return_tensors="pt")

output = model.generate(input_ids, max_length=300, num_return_sequences=1)

In [26]:
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

SELECT COUNT Data_atendimento FROM table WHERE Date = Dos_Qtde_Prescrita, Prontuario, data_at endimento = IDade_anos, idade_dias_1, procedimento_1) AND Date = Doctuario AND No.S. query only = no explanation                #### SQLite sql tables , with their properties = No explanation                                Prescriptions(Sqlite query only) = 0–0, 0–06, 0805–0–8, 0.06–9, AND IDade, 0 `` 0–9–0 = 0.5 AND 0–6–0.9 = 5–1–0 AND 0.005–9 = 1– AND 1–06–0 FROM table = 2–0 0– AND 2–5 = 0 AND 0 × = 0�–00–906 = 0�–0805–9 AND 0�–902 = 0�–976 AND 0�0–10 = 0�0–1, 0� AND 0�–977 = 0�–0 `` 0 `` 5 `` 0 0 `` 9 `` 0 AND0–6 = 0�–907 AND 0�0–07 = 0�co, 0�8 AND 0�–, 0�, 0�8 = 0quema, 0tde-esquema AND


In [ ]:
cursor.execute(f"SELECT COUNT(*) FROM {'Prescriptions'}")
result = cursor.fetchall()

print(result)

In [14]:
# connection.close()